In [9]:
from dotenv import load_dotenv

_ = load_dotenv()

from api.core.infisical import InfisicalManagedCredentials

secrets_client = InfisicalManagedCredentials()

from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
)
from api.services.embeddings_factory import EmbeddingsFactory
from api.services.vector_store_factory import VectorStoreFactory
from api.services.llm_factory import LLMFactory
from api.services.memory_factory import MemoryFactory
import api.config.constant as constant

2025-10-20 17:12:06 - api.core.infisical - INFO - Fetched secrets metadata from Infisical (values not logged)
2025-10-20 17:12:06 - api.core.infisical - INFO - Infisical Managed Credentials initialized


In [ ]:
embeddings = EmbeddingsFactory().get_embeddings(
    "sentence-transformers", "intfloat/multilingual-e5-large-instruct"
)

vector_store = VectorStoreFactory().get_vectorstore(
    vectorstore_service="astradb",
    embeddings=embeddings,
)

model = LLMFactory().get_chat_model(
    model_name="command-r-plus-08-2024",
)

2025-10-20 16:56:55 - api.services.embeddings_factory - INFO - Using Sentence Transformers embeddings model.
/home/subhadeepm/Documents/Education/Projects/Python/godot-rag-api/api/services/embeddings_factory.py:28: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return SentenceTransformerEmbeddings(
2025-10-20 16:56:59 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-large-instruct
2025-10-20 16:57:24 - api.services.vector_store_factory - INFO - Using AstraDB
2025-10-20 16:57:24 - langchain_astradb.vectorstores - INFO - vector store default init, collection 'godot_docs'
2025-10-20 16:57:25 

In [3]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [4]:
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=["context", "question"],
                input_types={},
                partial_variables={},
                template=constant.SYSTEM_PROMPT,
            ),
            additional_kwargs={},
        )
    ],
)

In [ ]:
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    session_memory = MemoryFactory().get_memory(
        memory_service="session",
        session_id=state.get("session_id", "default_session"),
    )
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = model.invoke(messages)
    return {"answer": response.content}

In [6]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
result = graph.invoke(
    {
        "question": "How to add 2D sprites in godot 4?",
        "session_id": "8237648732647238",
        "category": "tutorials",
        "sub_category": "None",
        "memory_service": "astradb",
        "top_k": 5,
        "model_name": "command-r-plus-08-2024",
    }
)

print(f"Context: {result['context']}")
print(f"Answer: {result['answer']}")

2025-10-20 17:13:34 - astrapy.data.cursors.cursor - INFO - cursor fetching a page: (empty page state) from godot_docs
2025-10-20 17:13:37 - astrapy.data.cursors.cursor - INFO - cursor finished fetching a page: (empty page state) from godot_docs


Context: [Document(id='6505200e9f3b4ce2b6273b4ee82375f8', metadata={'category': 'conversation', 'source': 'ImJimmeh/godot-training'}, page_content="# instruction\nWhat needs to be done to add more animations to a 2D sprite in Godot?\n# input\n# output\nTo add more animations to a 2D sprite: 1. Open the SpriteFrames resource in the editor. 2. Click the 'Add Animation' button. 3. Drag and drop additional images into the newly created animation slot. You can then control these animations via script similarly using `play()` and `stop()`. Example: ```gdscript\n@onready var _animated_sprite = $AnimatedSprite2D\nfunc play_animation(anim_name):\n    _animated_sprite.play(anim_name)\n```"), Document(id='32f621d511af4b59aaa4d14e43bf7ed8', metadata={'category': 'conversation', 'source': 'glaiveai/godot_4_docs'}, page_content="# prompt\nHow do you create and manipulate animations for a 2D sprite in Godot using AnimatedSprite2D?\n# response\nTo create and manipulate animations for a 2D sprite in Go

In [8]:
result

{'question': 'How to add 2D sprites in godot 4?',
 'context': [Document(id='6505200e9f3b4ce2b6273b4ee82375f8', metadata={'category': 'conversation', 'source': 'ImJimmeh/godot-training'}, page_content="# instruction\nWhat needs to be done to add more animations to a 2D sprite in Godot?\n# input\n# output\nTo add more animations to a 2D sprite: 1. Open the SpriteFrames resource in the editor. 2. Click the 'Add Animation' button. 3. Drag and drop additional images into the newly created animation slot. You can then control these animations via script similarly using `play()` and `stop()`. Example: ```gdscript\n@onready var _animated_sprite = $AnimatedSprite2D\nfunc play_animation(anim_name):\n    _animated_sprite.play(anim_name)\n```"),
  Document(id='32f621d511af4b59aaa4d14e43bf7ed8', metadata={'category': 'conversation', 'source': 'glaiveai/godot_4_docs'}, page_content="# prompt\nHow do you create and manipulate animations for a 2D sprite in Godot using AnimatedSprite2D?\n# response\nTo